In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib import data_processors, tasks
from lib.bert import BertForSequenceClassification
from lib.train_eval import train, evaluate, predict

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
%env CUDA_VISIBLE_DEVICES=0

params = {
    'data_dir': '../datasets/MRPC',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'mrpc',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 1,
    'seed': 1331,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])

env: CUDA_VISIBLE_DEVICES=1


In [3]:
params['num_labels'] = tasks.num_labels[params['task_name']]
params['label_list'] = tasks.label_lists[params['task_name']]

processor = tasks.processors[params['task_name']]()
tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
dev_examples = processor.get_dev_examples(params['data_dir'])

model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

## Training and evaluation

In [5]:
TOTAL_EPOCH_NUMS = 5

for epoch_num in range(1, TOTAL_EPOCH_NUMS + 1):
    params['num_train_epochs'] = 1
    checkpoint_files = {
        'config': 'bert_config.json',
        'file_to_save': 'model_{}_epoch_{}.pth'.format(
            params['task_name'], epoch_num)
    }

    model, result = train(model, tokenizer, params,
                          train_examples,
                          valid_examples=dev_examples,
                          checkpoint_files=checkpoint_files)

Iteration:   0%|          | 0/115 [00:00<?, ?it/s]

***** Running training *****
Num examples: 3668
Batch size:   32
Num steps:    114

Epoch: 1


Iteration: 100%|██████████| 115/115 [00:50<00:00,  2.44it/s]


{'train_loss': 0.5516028106212616, 'train_global_step': 115}
***** Running evaluation *****
Num examples:  408
Batch size:    8


Evaluating: 100%|██████████| 51/51 [00:01<00:00, 26.00it/s]


...

***** Running training *****
Num examples: 3668
Batch size:   32
Num steps:    114

Epoch: 1


Iteration: 100%|██████████| 115/115 [00:51<00:00,  2.44it/s]


{'train_loss': 0.13777643999327785, 'train_global_step': 115}
***** Running evaluation *****
Num examples:  408
Batch size:    8


Evaluating: 100%|██████████| 51/51 [00:01<00:00, 26.05it/s]


{'eval_loss': 0.6504761793837008, 'eval_accuracy': 0.8357843137254902, 'eval_f1_score': 0.882661996497373, 'eval_matthews_corrcoef': 0.6114202481916022}


In [9]:
best_model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

best_model.load_state_dict(torch.load(
    os.path.join(params['output_dir'], 'model_{}_epoch_{}.pth'.format(
        params['task_name'], 3))))

In [10]:
result, prob_preds = evaluate(best_model, tokenizer, params,
                              dev_examples)
result

***** Running evaluation *****
Num examples:  408
Batch size:    8


Evaluating: 100%|██████████| 51/51 [00:01<00:00, 26.16it/s]


{'eval_loss': 0.46879259559586095,
 'eval_accuracy': 0.8431372549019608,
 'eval_f1_score': 0.8892733564013842,
 'eval_matthews_corrcoef': 0.6258607092229798}